# Unit 5 - Financial Planning

In [ ]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation

%matplotlib inline

In [ ]:
# Load .env environment variables
load_dotenv()

# Check if API keys are loaded correctly
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

if not alpaca_api_key or not alpaca_secret_key:
    raise ValueError("API keys not found. Please check your .env file.")
else:
    print("API keys loaded successfully.")


## Part 1 - Personal Finance Planner

### Collect Crypto Prices Using the `requests` Library

In [ ]:
# Set current amount of crypto assets
my_btc = 1.2  # Example: owning 1.2 Bitcoins
my_eth = 5.3  # Example: owning 5.3 Ethereum


In [ ]:
# Crypto API URLs
btc_url = "https://api.alternative.me/v2/ticker/Bitcoin/?convert=USD"
eth_url = "https://api.alternative.me/v2/ticker/Ethereum/?convert=USD"

In [ ]:
# Initial import
import requests

# Fetch current BTC price
btc_response = requests.get(btc_url)
btc_data = btc_response.json()
btc_price = btc_data['data']['1']['quotes']['USD']['price']

# Fetch current ETH price
eth_response = requests.get(eth_url)
eth_data = eth_response.json()
eth_price = eth_data['data']['1027']['quotes']['USD']['price']

# Compute current value of my crpto
my_btc_value = my_btc * btc_price
my_eth_value = my_eth * eth_price

# Print current crypto wallet balance
print(f"The current value of your {my_btc} BTC is ${my_btc_value:0.2f}")
print(f"The current value of your {my_eth} ETH is ${my_eth_value:0.2f}")


### Collect Investments Data Using Alpaca: `SPY` (stocks) and `AGG` (bonds)

In [ ]:
# Set current amount of shares
my_agg = 200  # number of shares of AGG
my_spy = 50   # number of shares of SPY


In [ ]:
# Initial import
import os
from alpaca_trade_api import REST

# Set Alpaca API key and secret
# Set Alpaca API key and secret from environment variables
alpaca_api_key = os.getenv('ALPACA_API_KEY')
alpaca_secret_key = os.getenv('ALPACA_SECRET_KEY')

# Checking if the API key and secret are available
if alpaca_api_key is None or alpaca_secret_key is None:
    raise Exception("Alpaca API key or secret not found. Check your .env file.")

# Create the Alpaca API object
# Creating the Alpaca API object
alpaca = REST(alpaca_api_key, alpaca_secret_key, base_url="https://paper-api.alpaca.markets/v2", api_version='v2')


In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()  # This loads the environment variables from the .env file

# Fetching and printing to verify the keys are loaded (remove or comment out in production for security)
alpaca_api_key = os.getenv('ALPACA_API_KEY')
alpaca_secret_key = os.getenv('ALPACA_SECRET_KEY')

print("API Key:", alpaca_api_key)  # Should show your API key
print("Secret Key:", alpaca_secret_key)  # Should show your secret key



In [ ]:
# Initial import
import datetime
import pandas as pd
import pandas_market_calendars as mcal
import pytz

# Create a timezone-aware datetime object for 'now' in Eastern Time
eastern = pytz.timezone('America/New_York')
now = datetime.datetime.now(pytz.utc)

# Convert 'now' to Eastern Time and remove any time component
today = now.astimezone(eastern).replace(hour=0, minute=0, second=0, microsecond=0)

# Create a calendar for NYSE
nyse_calendar = mcal.get_calendar('NYSE')

# Get the last 10 valid trading days from today, adjusted for market timezone
valid_trading_days = nyse_calendar.valid_days(start_date=today - pd.DateOffset(days=10), end_date=today, tz='America/New_York')

# Find the last trading session (handles weekends and holidays)
last_trading_day = valid_trading_days[-1]

# Format the last trading day as ISO format
last_trading_date_iso = last_trading_day.date().isoformat()

print("Last trading day:", last_trading_date_iso)

# Set the tickers
tickers = ["AGG", "SPY"]

# Set timeframe to "1Day" for Alpaca API
timeframe = "1Day"

# Get current closing prices for SPY and AGG
portfolio_data = alpaca.get_bars(
    tickers,
    timeframe,
    start=last_trading_date_iso,
    end=last_trading_date_iso
).df

# Reorganize the DataFrame
agg_data = portfolio_data[portfolio_data['symbol'] == 'AGG'].drop(columns=["open", "high", "low", "volume", "symbol"])
spy_data = portfolio_data[portfolio_data['symbol'] == 'SPY'].drop(columns=["open", "high", "low", "volume", "symbol"])

# Concatenate the ticker DataFrames
combined_data = pd.concat([agg_data, spy_data], axis=1)
combined_data.columns = ["AGG_close", "SPY_close"]

# Preview DataFrame
print(combined_data.head())


In [ ]:
# Pick AGG and SPY close prices
# Checking if the DataFrame is not empty and has the necessary columns
if not combined_data.empty and {'AGG_close', 'SPY_close'}.issubset(combined_data.columns):
    # Pick AGG and SPY close prices
    agg_close_price = combined_data['AGG_close'].iloc[-1]  # Last entry's close price for AGG
    spy_close_price = combined_data['SPY_close'].iloc[-1]  # Last entry's close price for SPY
else:
    agg_close_price = "Unavailable"
    spy_close_price = "Unavailable"
    print("Data not available or DataFrame is not set up correctly.")

# Print AGG and SPY close prices
print(f"Current AGG closing price: ${agg_close_price}")
print(f"Current SPY closing price: ${spy_close_price}")

In [ ]:
# Compute the current value of shares
my_spy_value = my_spy * spy_close_price
my_agg_value = my_agg * agg_close_price

# Print current value of shares
print(f"The current value of your {my_spy} SPY shares is ${my_spy_value:0.2f}")
print(f"The current value of your {my_agg} AGG shares is ${my_agg_value:0.2f}")

### Savings Health Analysis

In [ ]:
# Set monthly household income
# YOUR CODE HERE!

# Consolidate financial assets data
# YOUR CODE HERE!

# Create savings DataFrame
# YOUR CODE HERE!

# Display savings DataFrame
display(df_savings)

In [ ]:
# Plot savings pie chart
# YOUR CODE HERE!

In [ ]:
# Set ideal emergency fund
emergency_fund = monthly_income * 3

# Calculate total amount of savings
# YOUR CODE HERE!

# Validate saving health
# YOUR CODE HERE!

## Part 2 - Retirement Planning

### Monte Carlo Simulation

In [ ]:
# Set start and end dates of five years back from today.
# Sample results may vary from the solution based on the time frame chosen
start_date = pd.Timestamp('2016-05-01', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2021-05-01', tz='America/New_York').isoformat()

In [ ]:
# Get 5 years' worth of historical data for SPY and AGG
# YOUR CODE HERE!

# Reorganize the DataFrame
# Separate ticker data
# YOUR CODE HERE!


# Concatenate the ticker DataFrames
# YOUR CODE HERE!

# Display sample data
df_stock_data.head()

In [ ]:
# Configuring a Monte Carlo simulation to forecast 30 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Printing the simulation input data
# YOUR CODE HERE!

In [ ]:
# Running a Monte Carlo simulation to forecast 30 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Plot simulation outcomes
# YOUR CODE HERE!

In [ ]:
# Plot probability distribution and confidence intervals
# YOUR CODE HERE!

### Retirement Analysis

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
# YOUR CODE HERE!

# Print summary statistics
# YOUR CODE HERE!

### Calculate the expected portfolio return at the `95%` lower and upper confidence intervals based on a `$20,000` initial investment.

In [ ]:
# Set initial investment
initial_investment = 20000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $20,000
# YOUR CODE HERE!

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower} and ${ci_upper}")

### Calculate the expected portfolio return at the `95%` lower and upper confidence intervals based on a `50%` increase in the initial investment.

In [ ]:
# Set initial investment
initial_investment = 20000 * 1.5

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $30,000
# YOUR CODE HERE!

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower} and ${ci_upper}")

## Optional Challenge - Early Retirement


### Five Years Retirement Option

In [ ]:
# Configuring a Monte Carlo simulation to forecast 5 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Running a Monte Carlo simulation to forecast 5 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Plot simulation outcomes
# YOUR CODE HERE!

In [ ]:
# Plot probability distribution and confidence intervals
# YOUR CODE HERE!

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
# YOUR CODE HERE!

# Print summary statistics
# YOUR CODE HERE!

In [ ]:
# Set initial investment
# YOUR CODE HERE!

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000
# YOUR CODE HERE!

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${ci_lower_five} and ${ci_upper_five}")

### Ten Years Retirement Option

In [ ]:
# Configuring a Monte Carlo simulation to forecast 10 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Running a Monte Carlo simulation to forecast 10 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Plot simulation outcomes
# YOUR CODE HERE!

In [ ]:
# Plot probability distribution and confidence intervals
# YOUR CODE HERE!

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
# YOUR CODE HERE!

# Print summary statistics
# YOUR CODE HERE!

In [ ]:
# Set initial investment
# YOUR CODE HERE!

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000
# YOUR CODE HERE!

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 10 years will end within in the range of"
      f" ${ci_lower_ten} and ${ci_upper_ten}")